In [ ]:
import pandas as pd
import numpy as np
import rasterio
import os


domain = "Rockies"
outputWorkspace = f"D:/ASOML/{domain}/modelOutputs/fromAlpine/"
testGroupWS = f"D:/ASOML/{domain}/test_groups/"
metaCSV = testGroupWS + "testGroupMetadata.csv"
aspect_CON = r"D:\ASOML\Rockies\features\ASO_CON_aspect_albn83_60m.tif"
elev_path = r"D:\ASOML\Rockies\features\ASO_CON_dem_albn83_60m.tif"
basemap = f"D:/ASOML/{domain}/basemap_data/"
features_binned = f"D:/ASOML/{domain}/features/binned/"
directionary_path = features_binned + "binned_raster_legends.csv"
interation = "20250717_090321"

# add in loop for iteration
model_runs = ["20250717_073937"]

for interation in model_runs:
    print(f"\n Model Run: {interation}") 
    csv_WS = outputWorkspace + f"{interation}/errorReview/"
    csv_path = csv_WS + f"{interation}_error_summary_stats.csv"
    
    # Load existing CSV
    df = pd.read_csv(csv_path)
    if 'mean_NSE' not in df.columns:
        df['mean_NSE'] = np.nan
    
    # Loop through each row/group
    nse_output_dir = outputWorkspace + f"{interation}/nse_rasters/"
    os.makedirs(nse_output_dir, exist_ok=True)
    meta_df = pd.read_csv(metaCSV)
    
    groups = ["G1", "G2", "G3", "G4", "G5", "G6"]
    # loop through each group
    for group in groups:
        print(group)
        
        # get valid raster
        group_meta  = meta_df[meta_df['GroupNum'] == f"{group}"]
        year = group_meta .iloc[0]['Year']
        test_doy = group_meta .iloc[0]['TestDOY']
        test_basin = group_meta .iloc[0]['TestBasin']
        validASO = f"D:/ASOML/{domain}/{year}/SWE_processed/{test_basin}_{test_doy}_albn83_60m_SWE.tif"
        model_raster = outputWorkspace + f"{interation}/outTifs_{group}_yPreds_tifs/mosaic_output/{interation}_{group}_cosine_mosaic_align.tif"
    
        with rasterio.open(model_raster) as src_model, rasterio.open(validASO) as src_val:
            model = src_model.read(1).astype(float)
            val = src_val.read(1).astype(float)
            
            mask = np.ones_like(model, dtype=bool)
            mask &= model != -1  # Model raster nodata value
            if src_val.nodata is not None:
                mask &= val != src_val.nodata
            
            print('rasters read')
            mask &= np.isfinite(model) & np.isfinite(val)
            mask &= ~np.isnan(model) & ~np.isnan(val)
            
            if np.sum(mask) == 0:
                print(f"No valid pixels found for {group}")
                continue
            
            # Calculate error
            error = model - val
            max_abs_error = np.max(np.abs(error[mask]))
            
            if max_abs_error == 0:
                normalized_error = np.full_like(error, 0.0)  # Perfect match
                mean_nse = 0.0
            else:
                normalized_error = np.full_like(error, -9999, dtype=np.float32)
                normalized_error[mask] = error[mask] / max_abs_error
                mean_nse = np.mean(np.abs(normalized_error[mask]))
            
        # Save NSE raster
        nse_raster_path = os.path.join(nse_output_dir, f"{interation}_{group}_NSE.tif")
        nse_meta = src_model.meta.copy()
        nse_meta.update(dtype="float32", nodata=-9999)
        
        with rasterio.open(nse_raster_path, "w", **nse_meta) as dst:
            dst.write(normalized_error.astype(np.float32), 1)
        
        # Update the CSV with mean NSE
        df.loc[df['GroupNum'] == group, 'mean_NSE'] = mean_nse
    
    # Save updated CSV (overwrite in place)
    df.to_csv(csv_path, index=False)
    print("CSV updated and NSE rasters saved.")

In [3]:
# testing with function 
import sys
sys.path.append("D:/ASOML/SNOCONE")
import pandas as pd
import numpy as np
import rasterio
import os
from CNN_benchmarks import*

domain = "Rockies"
outputWorkspace = f"D:/ASOML/{domain}/modelOutputs/fromAlpine/"
testGroupWS = f"D:/ASOML/{domain}/test_groups/"
metaCSV = testGroupWS + "testGroupMetadata.csv"
aspect_CON = r"D:\ASOML\Rockies\features\ASO_CON_aspect_albn83_60m.tif"
elev_path = r"D:\ASOML\Rockies\features\ASO_CON_dem_albn83_60m.tif"
basemap = f"D:/ASOML/{domain}/basemap_data/"
features_binned = f"D:/ASOML/{domain}/features/binned/"
directionary_path = features_binned + "binned_raster_legends.csv"
interation = "20250717_090321"

# add in loop for iteration
model_runs = ["20250717_073937"]
groups = ["G1"]

for interation in model_runs:
    print(f"\n Model Run: {interation}")
    meta_df = pd.read_csv(metaCSV)
    csv_WS = outputWorkspace + f"{interation}/errorReview/"
    csv_path = csv_WS + f"{interation}_error_summary_stats.csv"
    
    # loop through the groups
    for group in groups:
        print(group)
        
        # get valid raster
        group_meta  = meta_df[meta_df['GroupNum'] == f"{group}"]
        year = group_meta .iloc[0]['Year']
        test_doy = group_meta .iloc[0]['TestDOY']
        test_basin = group_meta .iloc[0]['TestBasin']
        validASO = f"D:/ASOML/{domain}/{year}/SWE_processed/{test_basin}_{test_doy}_albn83_60m_SWE.tif"
        model_raster = outputWorkspace + f"{interation}/outTifs_{group}_yPreds_tifs/mosaic_output/{interation}_{group}_cosine_mosaic_align.tif"

        # make output raster nse folder 
        os.makedirs(f"{outputWorkspace}/{interation}/nse_rasters/", exist_ok=True)
        print('directory made')
        output_nse = f"{outputWorkspace}/{interation}/nse_rasters/CNN_{group}_NSE.tif"

        error_raster, stats_df = normalized_SWE_error(model_raster, validASO, output_nse)
        print('normalized swe raster created')

        print(stats_df)
    


 Model Run: 20250717_073937
G1
directory made
NSE raster saved to: D:/ASOML/Rockies/modelOutputs/fromAlpine//20250717_073937/nse_rasters/CNN_G1_NSE.tif
normalized swe raster created
       metric     value
0     min_NSE  0.000000
1    mean_NSE  0.025309
2  median_NSE  0.011919
3     max_NSE  1.000000
